In [3]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd 
import numpy as np
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

In [4]:
columns = ['Number', 'Player', 'Age', 'Position', 'Mins played', 'Appearances', 'Line Ups', 'Sub In', 'Sub Out', 'Sub On Bench', 'Goals', 'Asists', 'Yellow', 'Yellow 2nd', 'Red', 'Club', 'League', 'Country']

In [5]:
def createList(url):
    page = requests.get(url) #,proxies=proxyDict)
    s = re.search('submenu_dropdown(.*)yui-main', str(page.content)).group(1)
    soup = BeautifulSoup(s, 'html.parser')
    period_tags = soup.find_all(re.compile("(option)")) #, class_=re.compile("(number|position|name|shirtnumber)"))
    country = [str(i).split('/')[2] for i in period_tags]
    team = [str(i).split('/')[3] for i in period_tags]
   
    return  [i for i in zip(country, team)]

In [6]:
def compute(country, league, team):
    page = requests.get('http://int.soccerway.com/teams/'+country+'/'+team+'/squad/') #,proxies=proxyDict)
    s = re.search('table squad sortable(.*)Coach', str(page.content)).group(1)
    soup = BeautifulSoup(s, 'html.parser')
    period_tags = soup.find_all(re.compile("(td)"), class_=re.compile("(number|position|name|shirtnumber)"))
    list = [i.get_text() for i in period_tags] # if 'coach' not in str(i)]
    #del list[-1:]
    A = np.array(list)
    B = pd.DataFrame(np.reshape(A, (int(A.size/15), 15)))
    B['15'] = team.replace('-', ' ').upper()
    B['16'] = league.replace('-', ' ').upper()
    B['17'] = country.replace('-', ' ').upper()
    return B

In [7]:
def concat(clubs, league):
    df = compute(clubs[0][0], league, clubs[0][1])
    for team in range(1,len(clubs)):
        df = pd.concat([df, compute(clubs[team][0], league, clubs[team][1])])
    return df

In [8]:
def type_changer(df):
    columns = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    for num in columns:
        if '' in df[num].unique():
            df = df[df[num] != '']
        df[num] = df[num].astype(int)
    return df

In [9]:
def column_changer(df):
    df.columns = columns
    return df

In [32]:
leagues = [('southampton-fc', 'PREMIER LEAGUE'),
           ('leeds-united-afc', 'CHAMPIONSHIP')]

In [33]:
def create_df(leagues):
    df = column_changer(type_changer(concat(createList('http://int.soccerway.com/teams/england/'+leagues[0][0]+'/squad/'), leagues[0][1])))
    for i in range(1,len(leagues)):
        df = pd.concat([df,column_changer(type_changer(concat(createList('http://int.soccerway.com/teams/england/'+leagues[i][0]+'/squad/'), leagues[i][1])))])
    return df

In [34]:
dff = create_df(leagues)

In [36]:
dff.head()

,Number,Player,Age,Position,Mins played,Appearances,Line Ups,Sub In,Sub Out,Sub On Bench,Goals,Asists,Yellow,Yellow 2nd,Red,Club,League,Country
0,1,A. Boruc,37,G,3060,34,34,0,0,0,0,0,2,0,0,AFC BOURNEMOUTH,PREMIER LEAGUE,ENGLAND
1,21,R. Allsop,24,G,0,0,0,0,0,16,0,0,0,0,0,AFC BOURNEMOUTH,PREMIER LEAGUE,ENGLAND
2,23,A. Federici,32,G,180,2,2,0,0,20,0,0,1,0,0,AFC BOURNEMOUTH,PREMIER LEAGUE,ENGLAND
3,,A. Ramsdale,18,G,0,0,0,0,0,0,0,0,0,0,0,AFC BOURNEMOUTH,PREMIER LEAGUE,ENGLAND
4,2,S. Francis,32,D,2761,32,32,0,2,0,0,1,3,0,1,AFC BOURNEMOUTH,PREMIER LEAGUE,ENGLAND


# England

#### Premier League

In [25]:
df_premier_league = createList('http://int.soccerway.com/teams/england/chelsea-football-club/661/squad/')

In [27]:
df_premier_league

[('england', 'afc-bournemouth'),
 ('england', 'arsenal-fc'),
 ('england', 'burnley-fc'),
 ('england', 'chelsea-football-club'),
 ('england', 'crystal-palace-fc'),
 ('england', 'everton-football-club'),
 ('england', 'hull-city-afc'),
 ('england', 'leicester-city-fc'),
 ('england', 'liverpool-fc'),
 ('england', 'manchester-city-football-club'),
 ('england', 'manchester-united-fc'),
 ('england', 'middlesbrough-football-club'),
 ('england', 'southampton-fc'),
 ('england', 'stoke-city-fc'),
 ('england', 'sunderland-association-football-club'),
 ('wales', 'swansea-city-afc'),
 ('england', 'tottenham-hotspur-football-club'),
 ('england', 'watford-football-club'),
 ('england', 'west-bromwich-albion-football-club'),
 ('england', 'west-ham-united-fc')]

#### Championship

In [29]:
df_championship = createList('http://int.soccerway.com/teams/england/brighton--hove-albion-fc/703/squad/')

In [30]:
df_championship

[('england', 'aston-villa-football-club'),
 ('england', 'barnsley-fc'),
 ('england', 'birmingham-city-fc'),
 ('england', 'blackburn-rovers-football-club'),
 ('england', 'brentford-fc'),
 ('england', 'brighton--hove-albion-fc'),
 ('england', 'bristol-city-fc'),
 ('england', 'burton-albion-fc'),
 ('wales', 'cardiff-city-fc'),
 ('england', 'derby-county-fc'),
 ('england', 'fulham-football-club'),
 ('england', 'huddersfield-town-fc'),
 ('england', 'ipswich-town-fc'),
 ('england', 'leeds-united-afc'),
 ('england', 'newcastle-united-football-club'),
 ('england', 'norwich-city-fc'),
 ('england', 'nottingham-forest-fc'),
 ('england', 'preston-north-end-fc'),
 ('england', 'queens-park-rangers-fc'),
 ('england', 'reading-fc'),
 ('england', 'rotherham-united-fc'),
 ('england', 'sheffield-wednesday-fc'),
 ('england', 'wigan-athletic-football-club'),
 ('england', 'wolverhampton-wanderers-fc')]

#### League One

In [38]:
df_league_one = createList('http://int.soccerway.com/teams/england/afc-wimbledon/4593/squad/')
df_league_one

[('england', 'afc-wimbledon'),
 ('england', 'bolton-wanderers-football-club'),
 ('england', 'bradford-city-afc'),
 ('england', 'bristol-rovers-fc'),
 ('england', 'bury-fc'),
 ('england', 'charlton-athletic-football-club'),
 ('england', 'chesterfield-fc'),
 ('england', 'coventry-city-fc'),
 ('england', 'fleetwood-town-fc'),
 ('england', 'gillingham-fc'),
 ('england', 'millwall-fc'),
 ('england', 'milton-keynes-dons-fc'),
 ('england', 'northampton-town-fc'),
 ('england', 'oldham-athletic-afc'),
 ('england', 'oxford-united-fc'),
 ('england', 'peterborough-united-fc'),
 ('england', 'port-vale-fc'),
 ('england', 'rochdale-afc'),
 ('england', 'scunthorpe-united-fc'),
 ('england', 'sheffield-united-fc'),
 ('england', 'shrewsbury-town-fc'),
 ('england', 'southend-united-fc'),
 ('england', 'swindon-town-fc'),
 ('england', 'walsall-fc')]

#### League Two

In [39]:
df_league_two = createList('http://int.soccerway.com/teams/england/plymouth-argyle-fc/701/')
df_league_two

[('england', 'accrington-stanley-fc'),
 ('england', 'barnet-fc'),
 ('england', 'blackpool-fc'),
 ('england', 'cambridge-united-fc'),
 ('england', 'carlisle-united-fc'),
 ('england', 'cheltenham-town-fc'),
 ('england', 'colchester-united-fc'),
 ('england', 'crawley-town-football-club'),
 ('england', 'crewe-alexandra-fc'),
 ('england', 'doncaster-rovers-fc'),
 ('england', 'exeter-city-fc'),
 ('england', 'grimsby-town-fc'),
 ('england', 'hartlepool-united-fc'),
 ('england', 'leyton-orient-fc'),
 ('england', 'luton-town-fc'),
 ('england', 'mansfield-town-fc'),
 ('england', 'morecambe-fc'),
 ('wales', 'newport-county-afc'),
 ('england', 'notts-county-fc'),
 ('england', 'plymouth-argyle-fc'),
 ('england', 'portsmouth-football-club'),
 ('england', 'stevenage-borough-fc'),
 ('england', 'wycombe-wanderers-fc'),
 ('england', 'yeovil-town-fc')]

In [14]:
df_england = pd.concat([df_premier_league,df_championship,df_league_one,df_league_two])

In [15]:
len(df_england['Club'].unique())

92

# Germany

#### Bundesliga

#### Bundesliga 2

# Merge and Export

In [16]:
#pd.concat([df_premier_league,df_bundesliga]).to_csv('premier_league.csv')